# Automate HCPCS Formating for Salesforce

Every quarter, the business requests HCPCS and ICD10 Codes to be updated in the master list for **Prism**, **Que**, **Salesforce**, and **Essette**. This script is used to format files to update and create new HCPCS in Salesforce.

This script will format the file given by the business into a format that is acceptable by Salesforce.

## Prerequisites
- No prior knowledge of Python is required to run this script. However, that knowledge will help you understand the logic and syntax of the script. Specifically, we will be using the **pandas** library to transpose the file given by the business to a DataFrame, then we will use that DataFrame to create a new DataFrame with the necessary information. We will also be using the **datetime** library specifically for naming conventions of our file.
- You will need to **rename** the file given by the business to **request.csv**. If this step is not done, then this script **will not run** properly.
- You will need to save this Jupyter Notebook and the file **request.csv** in the same directory, e.g. in the same location on your local machine.
- You will need a method to run Jupyter Notebooks on your local machine. This script assumes Anaconda has already been installed to your local machine. If Anaconda is not installed in your local machine, then you will need to install it via https://www.anaconda.com/docs/getting-started/anaconda/install.

## Overview of the Process
This script will read the files **request.csv** and **salesforce.csv** and load them into a DataFrame using the pandas library. 

Since Salesforce only cares about the following columns, we will also strip the unnecessary columns from **request**. The columns we will be using from **request** are
- **HCPCS**
- **Long Description**
- **Short Description**

We will be using all columns from **salesforce**. After we read the files, we will then join **salesforce** onto **request** via the **HCPCS** column. Finally, we will compare columns to find
1. HCPCS that need to be created in Salesforce
2. HCPCS that need to have Long Description updated in Salesforce
3. HCPCS that need to have Short Description updated in Salesforce
4. HCPCS that need to be marked as Inactive in Salesforce
5. HPCCS that need no changes 

Everything will be outputed to their own separate .csv files.

In [2]:
# Import Libraries
import pandas as pd
from datetime import datetime

today = datetime.today() # Get Today's Date
formatted_date = today.strftime("%m%d%Y") # Strip Today's Date in the MMDDYYYY Format

In [3]:
# Load the CSV files into a DataFrame
request    = pd.read_csv("salesforce-request-12-22-25.csv",    dtype=str) # Replace with your actual file name
salesforce = pd.read_csv("salesforce-prod-dec-2025.csv",  dtype=str) # Replace with your actual file name

In [4]:
salesforce.head()

,ID,HCPC Code,Equipment Type,HCPC Product Description,Product Details,Created Date
0,a0n300000065C79,T4535,DME,Disposable liner/shield/guard/pad/undergarment...,Disposable liner/shield/pad,8/7/24
1,a0n300000065C7A,T4523,DME,"Adult sized disposable incontinence product, b...",Adult size brief/diaper lg,8/7/24
2,a0n300000065C7B,A4927,DME,"Gloves, non-sterile, per 100",Non-sterile gloves,8/7/24
3,a0n300000065C7D,A4259,DME,"Lancets, per box of 100",Lancets per box,8/7/24
4,a0n300000065C7E,A4554,DME,"Disposable underpads, all sizes",Disposable underpads,8/7/24


In [5]:
request.head()

,HCPCS,Product Type,HCPCS Product Category Parent,HCPCS Product Category Parent Code,HCPCS Product Category Child,HCPCS Product Category Child Code,Que Category,QUE Category Code,Long Description,Short Description,Frequency,Miscellaneous Code,Labor Code,Action,Transition Categories,CMS Categories,Jurisdiction,Code Start Date,Code End Date,Code Run Out Date
0,A0021,NDME,Uncategorized,UCTG,Uncategorized,UCTG,Uncategorized,UCTG,"Ambulance service, outside state per mile, tra...",Outside state ambulance serv,NaN,0,0,CREATE,NDME,NaN,PBM,10-Sep-96,NaN,NaN
1,A4206,DME,Medical Supplies,MDS,Miscellaneous Medical Supplies,MMS,Uncategorized,UCTG,"Syringe with needle, sterile, 1 cc or less, each",1 cc sterile syringe&needle,NaN,0,0,CREATE,RO,SU,D,1-Jan-08,NaN,NaN
2,A0080,NDME,Uncategorized,UCTG,Uncategorized,UCTG,Uncategorized,UCTG,"Non-emergency transportation, per mile - vehic...",Noninterest escort in non er,NaN,0,0,CREATE,NDME,NaN,PBM,1-Jan-03,NaN,NaN
3,A4207,DME,Medical Supplies,MDS,Miscellaneous Medical Supplies,MMS,Uncategorized,UCTG,"Syringe with needle, sterile 2 cc, each",2 cc sterile syringe&needle,NaN,0,0,CREATE,RO,SU,D,1-Jan-07,NaN,NaN
4,A0090,NDME,Uncategorized,UCTG,Uncategorized,UCTG,Uncategorized,UCTG,"Non-emergency transportation, per mile - vehic...",Interest escort in non er,NaN,0,0,CREATE,NDME,NaN,PBM,1-Jan-03,NaN,NaN


```
# Select only the columns needed from request 
request = request[["HCPCS", "Long Description", "Short Description", "Product Type"]]
# Rename the columns of salesforce
salesforce.rename(columns={
    "HCPC: ID": "Salesforce ID",
    "HCPC": "HCPCS",
    "Type": "Type",
    "HCPC Product Long Description": "Long Description",
    "Product Details- Short": "Short Description"
}, inplace=True)

# New file provided had different column names; Creating new version (September 30, 2025)
# Select only the columns needed from request
request = request[["HCPCS", "Long Description", "Short Description", "Product Type"]]
# Rename the columns of salesforce
salesforce.rename(columns={
    "ID": "Salesforce ID",
    "DME Code": "HCPCS",
    "Equipment Type": "Type",
    "HCPC Product Description": "Long Description",
    "Product Details": "Short Description"
}, inplace=True)

# New file provided had different column names; Creating new version (September 30, 2025)
# Select only the columns needed from request
request = request[["HCPCS", "Long Description", "Short Description", "Product Type"]]
# Rename the columns of salesforce
salesforce.rename(columns={
    "ID": "Salesforce ID",
    "HCPC Code": "HCPCS",
    "Equipment Type": "Type",
    "HCPC Product Description": "Long Description",
    "Product Details": "Short Description"
}, inplace=True)
```

In [7]:
# Select only the columns needed from request 
request = request[["HCPCS", "Long Description", "Short Description", "Product Type"]]

# Standardize columns of Salesforce
salesforce.columns = ["ID", "HCPC Code", "Equipment Type", "HCPC Product Description", "Product Details", "Created Date"]
# Rename the columns of salesforce
salesforce.rename(columns={
    "ID": "Salesforce ID",
    "HCPC Code": "HCPCS",
    "Equipment Type": "Type",
    "HCPC Product Description": "Long Description",
    "Product Details": "Short Description",
    "Created Date": "Created Date"
}, inplace=True)



In [8]:
# Merge on HCPCS
merged = request.merge(salesforce, on = "HCPCS", how = "left", suffixes = ('_request', '_sf'))

# Find HCPCS Not in Salesforce
new_hcpcs = merged[merged["Salesforce ID"].isna()].copy()

# Find HCPCS that need to have Long Description updated in Salesforce
long_mismatch = merged[merged["Salesforce ID"].notna() &
                        (merged["Long Description_request"] != merged["Long Description_sf"])].copy()
long_mismatch = long_mismatch[["Salesforce ID", "HCPCS", "Type", "Long Description_request", "Long Description_sf"]]

# Find HCPCS that need to have Short Description updated in Salesforce
short_mismatch = merged[merged["Salesforce ID"].notna() &
                        (merged["Short Description_request"] != merged["Short Description_sf"]).copy()]
short_mismatch = short_mismatch[["Salesforce ID", "HCPCS", "Type", "Short Description_request", "Short Description_sf"]]

# Find HCPCS that need NO updates in Salesforce
matches = merged[merged["Salesforce ID"].notna() &
                    (merged["Long Description_request"] == merged["Long Description_sf"]) &
                    (merged["Short Description_request"] == merged["Short Description_sf"])].copy()

# Find HCPCS that need to be removed from Salesforce
# Left join Salesforce onto Request (original)
remove = salesforce.merge(request, on="HCPCS", how="left", suffixes=('_sf', '_request'))

# Filter for HCPCS in Salesforce not found in Request (original)
remove = remove[remove["Long Description_request"].isna()].copy()

In [9]:
# Check via Print Statements
print("There are " + str(len(request)) + " HCPCS in the request by the business.")
print("There are " + str(len(salesforce)) + " HCPCS currently in Salesforce.")
print("New HCPCS: " + str(len(new_hcpcs)))
print("Long Update: " + str(len(long_mismatch)))
print("Short Update: " + str(len(short_mismatch)))
print("Remove: " + str(len(remove)))
print("No Update: " + str(len(matches)))

There are 3792 HCPCS in the request by the business.
There are 3789 HCPCS currently in Salesforce.
New HCPCS: 9
Long Update: 133
Short Update: 5
Remove: 6
No Update: 3648


In [10]:
# Output the DataFrames to CSV Files
new_hcpcs.to_csv("New_HCPCS_" + formatted_date + ".csv", index = False)
long_mismatch.to_csv("Update_Long_Description_" + formatted_date + ".csv", index = False)
short_mismatch.to_csv("Update_Short_Description_" + formatted_date + ".csv", index = False)
remove.to_csv("Inactive_" + formatted_date + ".csv", index = False)
matches.to_csv("No_Changes_Needed_" + formatted_date + ".csv", index = False)